### demonstration of interactive acclimate data analysis

In [ ]:
%load_ext autoreload
%autoreload 2
from acclimate import dataset
from acclimate import definitions
from acclimate import helpers
from acclimate import plotting
import os
import numpy as np
import xarray as xr
import netCDF4
import seaborn as sb
import holoviews as hv
import param
import panel as pn
from holoviews.operation.datashader import datashade, dynspread
from holoviews.operation.timeseries import rolling, rolling_outlier_std
# define default renderers
hv.extension('bokeh',"matplotlib")
import math
golden = (1+math.sqrt(5))/2

#### import some data on consumption - using storages to look for substitution effects

Showing insane speed of dask chunked array, since aggregation working effortless in more or less real time.

In [3]:
output = dataset.AcclimateOutput("/home/quante/Documents/projects/acclimate_consumer/inequality_project/tmp/optim_global_fl_rcp-6p0_gcm-GFDL-ESM2M_hm-H08_fit-historical_2060-2070.nc")
storages = output.xarrays["storages"]
regions= output.xarrays["regions"]
production_sectors = range(0,26) # ignore consumption sectors

data without time dimension not loaded


In [4]:
consumer_storage_data = helpers.select_by_agent_properties(output.xarrays["storages"],output,type="consumer")

In [3]:
storages

,Array,Chunk
Bytes,7.59 GB,75.52 MB
Shape,"(4019, 31, 7613)","(40, 31, 7613)"
Count,102 Tasks,101 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,7.59 GB,75.52 MB
Shape,"(4019, 31, 7613)","(40, 31, 7613)"
Count,102 Tasks,101 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [4]:
aggregated_storage_data = helpers.aggregate_by_sector_group(storages, definitions.consumption_baskets)

In [5]:
aggregated_storage_data

,Array,Chunk
Bytes,734.32 MB,2.44 MB
Shape,"(3, 4019, 7613)","(1, 40, 7613)"
Count,2223 Tasks,303 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,734.32 MB,2.44 MB
Shape,"(3, 4019, 7613)","(1, 40, 7613)"
Count,2223 Tasks,303 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [6]:
i_region = "DEU"
%time specific_consumers = helpers.select_partial_data(helpers.select_by_agent_properties(output.xarrays["storages"],output,region=i_region,type="consumer"),sector=production_sectors)

CPU times: user 82.3 ms, sys: 107 µs, total: 82.4 ms
Wall time: 81.6 ms


In [7]:
specific_consumers

,Array,Chunk
Bytes,4.18 MB,41.60 kB
Shape,"(4019, 26, 5)","(40, 26, 5)"
Count,304 Tasks,101 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,4.18 MB,41.60 kB
Shape,"(4019, 26, 5)","(40, 26, 5)"
Count,304 Tasks,101 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [8]:
%time baseline_relative=aggregated_storage_data.map(dataset.baseline_relative)
baseline_relative

CPU times: user 33.1 ms, sys: 0 ns, total: 33.1 ms
Wall time: 32.9 ms


,Array,Chunk
Bytes,734.32 MB,2.44 MB
Shape,"(3, 4019, 7613)","(1, 40, 7613)"
Count,2535 Tasks,303 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,734.32 MB,2.44 MB
Shape,"(3, 4019, 7613)","(1, 40, 7613)"
Count,2535 Tasks,303 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [5]:
#get gdp of us regions to weight mean
outputfile = "/home/quante/Documents/projects/acclimate_consumer/inequality_project/tmp/core_consumer/consecutive_fixed_budget.nc"
data = dataset.AcclimateOutput(outputfile)
# calculate weight by gdp
def weighted_average_region_set(regions_to_aggregate):
    weights=[]
    for i_region in regions_to_aggregate:
        region_index = data.regions[i_region]
        weights.append(float(data.xarrays["regions"].sel(time=0,region=region_index)["gdp_value"].compute().data))
    weights = np.asarray(weights)
    normed_weights = weights/sum(weights)
    aggregated_data = []
    i=0
    for i_region in regions_to_aggregate:
        unweighted_data = (load_region_data(datadir,filename,i_region).clone(datatype=['xarray']).data)
        aggregated_data.append(unweighted_data*normed_weights[i])
        i+=1  
    all_data = xr.concat(aggregated_data, "region")
    mean = all_data.sum("region", skipna=True)
    return helpers.clean_vdims_consumption(hv.Dataset(mean),sector_map=definitions.basket_map)

data without time dimension not loaded


In [7]:
US_data = weighted_average_region_set(definitions.WORLD_REGIONS["USA"])

In [148]:
mean

<xarray.Dataset>
Dimensions:              (agent: 5, sector: 3, time: 2857)
Coordinates:
  * agent                (agent) int64 0 1 2 3 4
  * time                 (time) int64 0 1 2 3 4 5 ... 2852 2853 2854 2855 2856
  * sector               (sector) int64 0 1 2
Data variables:
    demand_quantity      (sector, time, agent) float64 1.0 1.0 ... 1.001 1.001
    demand_value         (sector, time, agent) float64 1.0 1.0 1.0 ... 1.0 1.0
    input_flow_quantity  (sector, time, agent) float64 1.0 1.0 1.0 ... 1.001 1.0
    input_flow_value     (sector, time, agent) float64 1.0 1.0 ... 0.9999 0.9996
    used_flow_quantity   (sector, time, agent) float64 1.0 1.0 ... 1.001 1.001
    used_flow_value      (sector, time, agent) float64 1.0 1.0 ... 0.998 0.9984

In [156]:
hv_mean

:Dataset   [agent,time,sector]   (demand_quantity,demand_value,input_flow_quantity,input_flow_value,used_flow_quantity,used_flow_value)

In [211]:
test = plotting.BoxplotExplorer(hv_mean)

## some experiments on linked selection

In [ ]:
# testing linked explorer:
class LinkedSelectionExplorer(plotting.VariableExplorer):
    @param.depends('rolling_window', 'variable', 'sector')
    def load_symbol(self):
        selection = self.dataset.select(sector=self.sector)
        curve = selection.to(hv.Curve, 'time', self.variable, groupby='agent').overlay()
        curve.opts(hv.opts.Curve(color=hv.Cycle(definitions.agent_colors)))

        distribution = selection.to(hv.Distribution, self.variable, [], groupby='agent').overlay()
        distribution.opts(hv.opts.Distribution(filled=True, color=hv.Cycle(definitions.agent_colors), alpha=0.9))

        test = curve+distribution
        return test

    @param.depends('rolling_window', 'variable', 'sector', 'agent')
    def view(self):
        dmap = hv.DynamicMap(self.load_symbol)
        # TODO:        # add rolling mean
        ls = link_selections.instance()
        return ls(dmap)

